In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
spam_df=pd.read_csv('../Downloads/spam.csv',encoding = "ISO-8859-1")

In [29]:
x_samp=spam_df.sample(10)


In [35]:
x_samp['v2']=x_samp['v2'].str.lower()

In [36]:
x_samp

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
921,ham,on ma way to school. can you pls send me ashle...,NaN,NaN,NaN
3432,ham,christmas is an occasion that is celebrated as...,NaN,NaN,NaN
1223,ham,rofl betta invest in some anti aging products,NaN,NaN,NaN
2782,ham,well at this right i'm gonna have to get up an...,NaN,NaN,NaN
5257,ham,"as usual..iam fine, happy &amp; doing well..:)",NaN,NaN,NaN
2807,ham,yes princess! i want to catch you with my big ...,NaN,NaN,NaN
5472,ham,well obviously not because all the people in m...,NaN,NaN,NaN
5070,ham,as in different styles?,NaN,NaN,NaN
2688,ham,yes i know the cheesy songs from frosty the sn...,NaN,NaN,NaN
3865,ham,"theoretically yeah, he could be able to come",NaN,NaN,NaN


In [37]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [38]:
cleanhtml('<h1>Ernest harnesser</h1><br/><a href="we.ho">link</a>')

'Ernest harnesserlink'